In [1]:
!pip install pymupdf nltk scikit-learn flask-ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 26.0 MB/s eta 0:00:00


In [2]:
import fitz
import re
import nltk
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

from flask import Flask,request,jsonify,render_template
from flask_ngrok import run_with_ngrok
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import files
uploaded=files.upload()
pdf_path=next(iter(uploaded))
def extract_text(pdf_file):
  doc=fitz.open(pdf_file)
  text=''
  for page in doc:
    text+=page.get_text()
    return text
raw_text=extract_text(pdf_path)

Saving Dataset final.pdf to Dataset final.pdf


In [4]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9.?! ]+', '', text)
    return text
def simple_summarize(text, num_words=30):
    words = text.split()
    summary = ' '.join(words[:num_words])
    if len(words) > num_words:
        summary += '...'
    return summary

In [5]:
def generate_study_plan(topics, days):
    plan = []
    topics_per_day = max(1, len(topics) // days)
    i = 0
    for day in range(1, days + 1):
        day_topics = topics[i:i + topics_per_day]
        if not day_topics:
            break
        plan.append(f"Day {day}: {', '.join(day_topics)}")
        i += topics_per_day
    return plan


In [6]:
html_code= """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8" />
    <title>Text Summarizer & Study Planner</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #e0f7fa;
            margin: 20px;
            color: #333;
        }
        h1, h3 {
            text-align: center;
            color: #00796b;
        }
        form {
            background: white;
            max-width: 600px;
            margin: 20px auto;
            padding: 15px 20px;
            border-radius: 8px;
            box-shadow: 0 0 8px rgba(0,0,0,0.1);
        }
        label {
            display: block;
            margin-top: 10px;
            font-weight: bold;
        }
        textarea, input[type="text"], input[type="number"] {
            width: 100%;
            padding: 8px;
            margin-top: 5px;
            border: 1px solid #00796b;
            border-radius: 5px;
            box-sizing: border-box;
            font-size: 1rem;
        }
        button {
            margin-top: 15px;
            background-color: #00796b;
            color: white;
            border: none;
            padding: 10px 20px;
            border-radius: 5px;
            cursor: pointer;
            font-size: 1rem;
        }
        button:hover {
            background-color: #004d40;
        }
        p, ul {
            max-width: 600px;
            margin: 15px auto;
            background: white;
            padding: 15px;
            border-radius: 8px;
            box-shadow: 0 0 8px rgba(0,0,0,0.1);
        }
        ul {
            list-style-type: disc;
            padding-left: 20px;
        }
        hr {
            max-width: 600px;
            margin: 40px auto;
            border: none;
            border-top: 2px solid #00796b;
        }
    </style>
</head>
<body>
    <h1>Text Summarizer</h1>
    <form action="/summarize" method="POST">
        <label for="text">Paste text to summarize:</label>
        <textarea name="text" id="text" rows="5" placeholder="Enter your text here"></textarea>

        <label for="num_words">Summary length (words):</label>
        <input type="number" name="num_words" id="num_words" value="30" min="10" max="100" />

        <button type="submit">Summarize</button>
    </form>

    {% if summary %}
        <h3>Summary:</h3>
        <p>{{ summary }}</p>
    {% endif %}

    <hr />

    <h1>Study Plan Generator</h1>
    <form action="/plan" method="POST">
        <label for="topics">Enter topics (comma separated):</label>
        <input type="text" id="topics" name="topics" placeholder="Math, History, Science" />

        <label for="days">Number of days:</label>
        <input type="number" id="days" name="days" value="7" min="1" max="30" />

        <button type="submit">Generate Study Plan</button>
    </form>

    {% if study_plan %}
        <h3>Your Study Plan:</h3>
        <ul>
            {% for item in study_plan %}
                <li>{{ item }}</li>
            {% endfor %}
        </ul>
    {% endif %}
</body>
</html>


"""
import os
os.makedirs("templates", exist_ok=True)
with open("templates/index.html", "w") as f:
    f.write(html_code)

In [ ]:
!pip install flask pyngrok nltk --quiet
from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok
import re
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
ngrok.set_auth_token("33C5djExD84fX2O6MOHKh9HjXIh_qG5dxQv6MN61fzCqgbBL")
app = Flask(__name__)
@app.route('/', methods=['GET'])
def home():
    return render_template_string(html_code)

@app.route('/summarize', methods=['POST'])
def summarize_route():
    text = request.form.get('text', '')
    num_words = int(request.form.get('num_words', 30))
    summary = simple_summarize(text, num_words)
    return render_template_string(html_code, summary=summary)

@app.route('/plan', methods=['POST'])
def plan_route():
    topics_str = request.form.get('topics', '')
    days = int(request.form.get('days', 7))
    topics = [t.strip() for t in topics_str.split(',') if t.strip()]
    study_plan = generate_study_plan(topics, days)
    return render_template_string(html_code, study_plan=study_plan)

if __name__ == '__main__':

    public_url = ngrok.connect(5000)
print(f"🚀 Public URL: {public_url}")
app.run(port=5000)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


🚀 Public URL: NgrokTunnel: "https://slouchy-myriadly-abril.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
